In [1]:
import seaborn as sns
import numpy as np
import pandas as pd

### 1. IRIS

#### a. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등 기초통계량(describe())을 구하시오.

In [2]:
iris = sns.load_dataset("iris")

In [3]:
# .agg(['mean', 'std'])를 통해 원하는것만 뽑아 낼 수도 있다.
iris.groupby(iris.species).describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

##### b. 이상치에 관한 표준편차와 평균 구하기

In [28]:
def get_outlier(x):
    q1 = x.quantile(.25)
    q3 = x.quantile(.75)
    iqr = q3-q1
    lower = q1-1.5*iqr
    upper = q3+1.5*iqr
    x[(x<lower) | (x>upper)] = np.nan
    #outlier = x.value_counts().sum() != 50
    outiler = x.isnull().sum() != 0 #코드의 재사용을 위해서는 isnull쪽이 좋다.
    return outlier, x.mean(), x.std()

In [29]:
species = []
columns = []
o_mean_list = []
o_std_list = []
outlier_list = []
n_mean_list = []
n_std_list = []
for spec in ['setosa', 'versicolor', 'virginica']:
    for sp in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']:
        s = iris[iris.species == spec][sp]
        outlier, new_mean, new_std = get_outlier(s)

        species.append(spec)
        columns.append(sp)
        o_mean_list.append(s.mean())
        o_std_list.append(s.std())
        outlier_list.append(outlier)
        n_mean_list.append(new_mean)
        n_std_list.append(new_std)

In [30]:
pd.options.display.float_format = lambda iris2: f'{iris2:.2f}'
iris2 = pd.DataFrame({
    "species" : species,
    "columns" : columns,
    "old_mean" : o_mean_list,
    "old_std" : o_std_list,
    "outlier" : outlier_list,
    "new_mean" : n_mean_list,
    "new_std" : n_std_list
})
n_iris = iris2.set_index(['species', 'columns'])
n_iris

old_mean  old_std  outlier  new_mean  new_std
species    columns                                                    
setosa     sepal_length      5.01     0.35    False      5.01     0.35
           sepal_width       3.43     0.32    False      3.43     0.32
           petal_length      1.46     0.13    False      1.46     0.13
           petal_width       0.23     0.09    False      0.23     0.09
versicolor sepal_length      5.94     0.52    False      5.94     0.52
           sepal_width       2.77     0.31    False      2.77     0.31
           petal_length      4.29     0.44    False      4.29     0.44
           petal_width       1.33     0.20    False      1.33     0.20
virginica  sepal_length      6.62     0.59    False      6.62     0.59
           sepal_width       2.96     0.26    False      2.96     0.26
           petal_length      5.55     0.55    False      5.55     0.55
           petal_width       2.03     0.27    False      2.03     0.27

### 2. titanic

##### a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오.

In [7]:
titanic = sns.load_dataset('titanic')

In [8]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.00,891.00,714.00,891.00,891.00,891.00
mean,0.38,2.31,29.70,0.52,0.38,32.20
std,0.49,0.84,14.53,1.10,0.81,49.69
min,0.00,1.00,0.42,0.00,0.00,0.00
25%,0.00,2.00,20.12,0.00,0.00,7.91
50%,0.00,3.00,28.00,0.00,0.00,14.45
75%,1.00,3.00,38.00,1.00,0.00,31.00
max,1.00,3.00,80.00,8.00,6.00,512.33


In [9]:
titanic["category1"] = titanic.apply(lambda r: r.sex if r.age>=20 else "child", axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.00,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.00,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,nan,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.00,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.00,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


##### b. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치시오.

In [10]:
pd.options.display.float_format = lambda titanic: f'{titanic:.2f}'
titanic['age'] = titanic['age'].fillna(titanic['age'].mean())
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.00,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.00,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.70,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.00,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.00,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


#### c. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [11]:
# 성별별 생존률
titanic.pivot_table('survived', 'sex')

,survived
sex,
female,0.74
male,0.19


In [12]:
# 선실별 생존률
titanic.pivot_table('survived', 'class')

,survived
class,
First,0.63
Second,0.47
Third,0.24


In [13]:
# 출발지별 생존률
titanic.pivot_table('survived', 'embark_town')

,survived
embark_town,
Cherbourg,0.55
Queenstown,0.39
Southampton,0.34


##### d. 타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오.

In [14]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanic['age_cat'] = pd.cut(titanic["age"], bins, labels=labels)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False,male,청년
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,female,중년
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,female,청년
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False,female,중년
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True,male,중년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.00,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년
887,1,1,female,19.00,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자
888,0,3,female,29.70,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child,청년
889,1,1,male,26.00,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년


In [15]:
titanic.pivot_table('survived', 'age_cat')

,survived
age_cat,
미성년자,0.42
청년,0.33
중년,0.42
장년,0.36
노년,0.20


#### e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오

In [16]:
titanic["q_age"] = pd.qcut(titanic["age"], 3, labels=['one', 'two', 'three'])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat,q_age
886,0,2,male,27.00,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년,two
887,1,1,female,19.00,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자,one
888,0,3,female,29.70,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child,청년,two
889,1,1,male,26.00,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년,two
890,0,3,male,32.00,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년,three


In [17]:
# 나이 그룹별 남녀 성비
# 남자를 1로, 여자를 0으로
titanic['num_sex'] = titanic.apply(lambda x: 1 if x.sex == 'male' else 0, axis=1)
# pivot테이블 만들기
titanic.pivot_table('num_sex','q_age')

,num_sex
q_age,
one,0.59
two,0.68
three,0.67


In [18]:
# 나이 그룹별 남녀 생존률
titanic.pivot_table('survived', ['q_age', 'sex'])

survived
q_age sex             
one   female      0.71
      male        0.21
two   female      0.70
      male        0.16
three female      0.82
      male        0.20

### mpg

In [19]:
mpg = sns.load_dataset('mpg')
pd.options.display.float_format = lambda x: f'{x:.2f}'

#### a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오

In [20]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.00,398.00,398.00,392.00,398.00,398.00,398.00
mean,23.51,5.45,193.43,104.47,2970.42,15.57,76.01
std,7.82,1.70,104.27,38.49,846.84,2.76,3.70
min,9.00,3.00,68.00,46.00,1613.00,8.00,70.00
25%,17.50,4.00,104.25,75.00,2223.75,13.83,73.00
50%,23.00,4.00,148.50,93.50,2803.50,15.50,76.00
75%,29.00,8.00,262.00,126.00,3608.00,17.17,79.00
max,46.60,8.00,455.00,230.00,5140.00,24.80,82.00


In [21]:
mpg["hp_per_cc"] = mpg.horsepower / mpg.displacement
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
393,27.00,4,140.00,86.00,2790,15.60,82,usa,ford mustang gl,0.61
394,44.00,4,97.00,52.00,2130,24.60,82,europe,vw pickup,0.54
395,32.00,4,135.00,84.00,2295,11.60,82,usa,dodge rampage,0.62
396,28.00,4,120.00,79.00,2625,18.60,82,usa,ford ranger,0.66
397,31.00,4,119.00,82.00,2720,19.40,82,usa,chevy s-10,0.69


#### b. name으로부터 manufacturer(제조사)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

In [22]:
name_split = mpg['name'].str.split(' ')

manufacturer = name_split.str[0]
model = name_split.str[1] + ' ' + name_split.str[2].fillna('')

mpg['manufacturer'] = manufacturer
mpg['model'] = model

mpg.drop(['name'], axis='columns', inplace=True)

mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.00,8,307.00,130.00,3504,12.00,70,usa,0.42,chevrolet,chevelle malibu
1,15.00,8,350.00,165.00,3693,11.50,70,usa,0.47,buick,skylark 320
2,18.00,8,318.00,150.00,3436,11.00,70,usa,0.47,plymouth,satellite
3,16.00,8,304.00,150.00,3433,12.00,70,usa,0.49,amc,rebel sst
4,17.00,8,302.00,140.00,3449,10.50,70,usa,0.46,ford,torino
...,...,...,...,...,...,...,...,...,...,...,...
393,27.00,4,140.00,86.00,2790,15.60,82,usa,0.61,ford,mustang gl
394,44.00,4,97.00,52.00,2130,24.60,82,europe,0.54,vw,pickup
395,32.00,4,135.00,84.00,2295,11.60,82,usa,0.62,dodge,rampage
396,28.00,4,120.00,79.00,2625,18.60,82,usa,0.66,ford,ranger


#### c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오

In [23]:
mpg.pivot_table('mpg', 'cylinders')

,mpg
cylinders,
3,20.55
4,29.29
5,27.37
6,19.99
8,14.96


#### d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

In [24]:
mpg[['origin', 'hp_per_cc']].groupby('origin').mean()

,hp_per_cc
origin,
europe,0.74
japan,0.79
usa,0.51


#### e. 모델이 5개 이상인 제조사에 대하여 연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오

In [25]:
# 모델이 5개인 회사
a = mpg.manufacturer.value_counts()
over_five = a[a>=5].index.tolist()

In [26]:
mpg[mpg['manufacturer'].isin(over_five)][['manufacturer','mpg']].groupby('manufacturer').mean().sort_values(by='mpg', ascending=False)[:5]

,mpg
manufacturer,
vw,39.02
honda,33.76
renault,32.88
datsun,31.11
mazda,30.86
